# Coste

Callback just in OpenAi

In [48]:
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(name="gpt-3.5-turbo", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

Tokens Used: 36
	Prompt Tokens: 4
	Completion Tokens: 32
Successful Requests: 1
Total Cost (USD): $7e-05


In [1]:
from langchain_openai import OpenAI

In [2]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [4]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm.invoke(text))



Day 1:
- Warm up with a 5-10 minute jog or brisk walk
- 10 minute run at a moderate pace
- 5 minute sprint interval training (30 seconds sprint, 30 seconds rest)
- 10 minute run at a moderate pace
- 5 minute cool down walk
- Stretching for 10-15 minutes

Day 2:
- Warm up with a 5-10 minute jog or brisk walk
- 20 minute bike ride or stationary bike at a moderate pace
- 5 minute hill sprints (30 seconds sprint up a hill, 30 seconds rest)
- 20 minute bike ride or stationary bike at a moderate pace
- 5 minute cool down walk
- Stretching for 10-15 minutes

Day 3:
- Warm up with a 5-10 minute jog or brisk walk
- 30 minute hike or trail run at a steady pace
- 5 minute bodyweight circuit (squats, lunges, push-ups, planks)
- 30 minute hike or trail run at a steady pace
- 5 minute cool down walk
- Stretching for 10-15 minutes

Day 4:
- Rest day or light activity such as yoga


In [9]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
ans = chain.invoke(input={"product": "eco-friendly water bottles"})
print(ans['text'])

In [18]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)


# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Well, I am an artificial intelligence program designed and created by a team of programmers. I am constantly learning and improving my abilities through algorithms and data analysis. My main purpose is to assist and provide information to users like yourself. I a

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

In [7]:
# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# Using VECTOR DB

In [8]:
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
# Add activeloop variable that is in .env file

import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
act_key = os.getenv("ACTIVELOOP_TOKEN")

os.environ["ACTIVELOOP_TOKEN"] = act_key


my_activeloop_org_id = "e2tovar"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding =embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://e2tovar/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:11<00:00, 11.35s/it]

Dataset(path='hub://e2tovar/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['a4096493-bfbc-11ee-b60b-482ae342e600',
 'a4096494-bfbc-11ee-98a7-482ae342e600']

Now, let's create a RetrievalQA chain:

In [9]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

Next, let's create an agent that uses the RetrievalQA chain as a tool:

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

c:\Users\Eddy.Tovar\Documents\Datos\work\PycharmProjects\proyectos_personales\cursos_llm\activeloop_langchain_vector\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new AgentExecutor chain...
 I should use the Retrieval QA System to find the answer
Action: Retrieval QA System
Action Input: "When was Napoleone born?"
Observation: 
Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


Let’s add an example of reloading an existing vector store and adding more data.

In [14]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embeddings=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://e2tovar/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:11<00:00, 11.34s/it]

Dataset(path='hub://e2tovar/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
   text       text      (6, 1)      str     None   


['3f1a0828-bfbd-11ee-b082-482ae342e600',
 '3f1a0829-bfbd-11ee-b8f9-482ae342e600']

We then recreate our previous agent and ask a question that can be answered only by the last documents added.



In [15]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [16]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new AgentExecutor chain...
 I should use the Retrieval QA System to find the answer.
Action: Retrieval QA System
Action Input: "When was Michael Jordan born?"
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


In [17]:
response

'Michael Jordan was born on 17 February 1963.'

 # Agent will use the Google Search tool to look up recent information about the Mars rover

In [20]:
from langchain_openai import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()
google_key = os.environ.get("GOOGLE_API_KEY")
cse_key = os.environ.get("GOOGLE_CSE_ID")

os.environ["GOOGLE_CSE_ID"] = cse_key
os.environ["GOOGLE_API_KEY"] = google_key

In [21]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

We can now define the Google search wrapper as follow.



In [25]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

- name parameter: This is a string that serves as a unique identifier for the tool. In this case, the name of the tool is "google-search.”
- func parameter: This parameter is assigned the function that the tool will execute when called. In this case, it's the run method of the search object, which presumably performs a Google search.
- description parameter: This is a string that briefly explains what the tool does. The description explains that this tool is helpful when you need to use Google to answer questions about current events.

In [26]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

Next, we create an agent that uses our Google Search tool:

- initialize_agent(): This function call creates and initializes an agent. An agent is a component that determines which actions to take based on user input. These actions can be using a tool, returning a response to the user, or something else.
- tools:  represents the list of Tool objects that the agent can use.
- agent="zero-shot-react-description": The "zero-shot-react-description" type of an Agent uses the ReAct framework to decide which tool to use based only on the tool's description.
- verbose=True: when set to True, it will cause the Agent to print more detailed information about what it's doing. This is useful for debugging and understanding what's happening under the hood.
- max_iterations=6: sets a limit on the number of iterations the Agent can perform before stopping. It's a way of preventing the agent from running indefinitely in some cases, which may have unwanted monetary costs.

In [27]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

In [30]:
response = agent.invoke("What's the latest news about the Mars rover?")
print(response['output'])



> Entering new AgentExecutor chain...
 I should search for news articles about the Mars rover.
Action: google-search
Action Input: "Mars rover news"
Observation: The Latest from Mars ... Videos from the rover show its shadow moving across the Martian surface during a 12-hour sequence while Curiosity remained parked. More ... Apr 25, 2023 ... The fully robotic Zhurong, named after a mythical Chinese god of fire, was expected to have woken up in December after entering a planned sleep ... NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration. Oct 19, 2022 ... ... Curiosity, visit: http://mars.nasa.gov/msl. Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. News Media Contact. Andrew Good. Jet ... A pair of quakes in 2021 sent seismic waves deep into the Red Planet's core, giving scientists the best data yet on its size and composition. ... Ten sample tubes ... Dec 15, 2021 ... These and o

In next example, two tools are being defined for use within a LangChain agent: a Google Search tool and a Language Model tool acting specifically as a text summarizer. The Google Search tool, using the GoogleSearchAPIWrapper, will handle queries that involve finding recent event information. The Language Model tool leverages the capabilities of a language model to summarize texts. These tools are designed to be used interchangeably by the agent, depending on the nature of the user's query.

In [32]:
from langchain_openai import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

Then instantiate a  LLMChain specifically for text summarization.

In [33]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

Next, we create the tools that our agent will use.

In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

We are now ready to create our agent that leverages two tools.

In [44]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [46]:
response = agent.invoke("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new AgentExecutor chain...
 I should search for the latest news about the Mars rover.
Action: google-search
Action Input: "Mars rover news"
Observation: The Latest from Mars ... Videos from the rover show its shadow moving across the Martian surface during a 12-hour sequence while Curiosity remained parked. More ... Apr 25, 2023 ... The fully robotic Zhurong, named after a mythical Chinese god of fire, was expected to have woken up in December after entering a planned sleep ... NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration. Oct 19, 2022 ... ... Curiosity, visit: http://mars.nasa.gov/msl. Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. News Media Contact. Andrew Good. Jet ... A pair of quakes in 2021 sent seismic waves deep into the Red Planet's core, giving scientists the best data yet on its size and composition. ... Ten sample tubes ... Dec 15, 2021 ... These and